In [1]:
import os
os.chdir('..')

In [2]:
import inspect
import multiprocessing as mp
from typing import List, Tuple

import numba as nb
import numpy as np
import pandas as pd
import time
from matplotlib import pyplot as plt

from neo_backtesting import Backtester, run_gridsearch
from strategy import boll_inbar
from util import read_candle_feather, transform_candle_np_struct, transform_np_struct
from joblib import Parallel, delayed

from xbx import equity_curve_for_OKEx_USDT_future_next_open, signal_simple_bolling

%matplotlib inline

In [3]:
# 回测起始日
START_DATE = '20180301'
END_DATE = '20220901'

# 策略参数
N = 100
B = 1.8
LONG_INTERVAL = '1h'

LEVERAGE = 1


# 回测参数
INIT_CAPITAL = 1e5  # 初始资金，10万
FACE_VALUE = 0.001  # 合约面值 0.001

COMM_RATE = 6e-4  # 交易成本万分之 6
LIQUI_RATE = 5e-3  # 爆仓保证金率千分之 5

CONTRACT_TYPE = 'futures'  # 正向合约

# 模拟器参数
SIMULATOR_PARAMS = {
    'init_capital': INIT_CAPITAL, 
    'face_value': FACE_VALUE, 
    'comm_rate': COMM_RATE, 
    'liqui_rate': LIQUI_RATE, 
    'init_pos': 0
}

STRA = boll_inbar  # 要回测的策略

ETH_PATHS = {
    '1m': '/home/lostleaf/feather_data/spot/ETH-USDT_1m.fea',
    '30m': '/home/lostleaf/feather_data/spot/ETH-USDT_30m.fea',
    '1h': '/home/lostleaf/feather_data/spot/ETH-USDT_1h.fea'
}

In [16]:
%time backtester = Backtester(ETH_PATHS, 'futures', SIMULATOR_PARAMS, boll_inbar)

factor_params = {
    'n': N,
    'b': B,
    'itl': LONG_INTERVAL
}

strategy_params = {
    'leverage': LEVERAGE
}

%time df_ret = backtester.run_detailed(START_DATE, END_DATE, INIT_CAPITAL, FACE_VALUE, factor_params, strategy_params)
df_ret['equity'] /= df_ret['equity'].iat[0]
print(df_ret[['candle_begin_time', 'close', 'pos', 'equity']])

CPU times: user 575 ms, sys: 610 ms, total: 1.19 s
Wall time: 513 ms
CPU times: user 1.26 s, sys: 694 ms, total: 1.96 s
Wall time: 1.9 s
          candle_begin_time    close       pos      equity
275636  2018-03-01 00:00:00   853.00         0    1.000000
275637  2018-03-01 00:01:00   852.80         0    1.000000
275638  2018-03-01 00:02:00   853.01         0    1.000000
275639  2018-03-01 00:03:00   852.97         0    1.000000
275640  2018-03-01 00:04:00   851.00         0    1.000000
...                     ...      ...       ...         ...
2638630 2022-08-31 23:56:00  1553.80  15620134  242.558917
2638631 2022-08-31 23:57:00  1554.28  15620134  242.633894
2638632 2022-08-31 23:58:00  1555.01  15620134  242.747921
2638633 2022-08-31 23:59:00  1554.10  15620134  242.605778
2638634 2022-09-01 00:00:00  1553.04  15620134  242.440204

[2362999 rows x 4 columns]


In [11]:
def load_candles(candle_paths):
    return {itl: read_candle_feather(path) for itl, path in candle_paths.items()}

def backtest(candles, factor_params, leverage):
    df_xbx = STRA.factor(candles, factor_params)
    df_xbx = df_xbx[(df_xbx['candle_begin_time'] >= START_DATE) & (df_xbx['candle_begin_time'] <= END_DATE)]
    df_xbx = signal_simple_bolling(df_xbx)
    df_xbx['pos'] = df_xbx['signal'].shift().ffill().fillna(0)
    df_xbx = equity_curve_for_OKEx_USDT_future_next_open(
        df_xbx,
        slippage=0,
        c_rate=COMM_RATE,
        leverage_rate=leverage,
        face_value=FACE_VALUE,
        min_margin_ratio=LIQUI_RATE)
    return df_xbx

%time candles = load_candles(ETH_PATHS)
%time df_xbx = backtest(candles, factor_params, LEVERAGE)
print(df_xbx[['candle_begin_time', 'close', 'pos', 'equity_curve']])

CPU times: user 404 ms, sys: 545 ms, total: 949 ms
Wall time: 321 ms
CPU times: user 3.42 s, sys: 2.55 s, total: 5.97 s
Wall time: 5.41 s
          candle_begin_time    close  pos  equity_curve
275636  2018-03-01 00:00:00   853.00  0.0      1.000000
275637  2018-03-01 00:01:00   852.80  0.0      1.000000
275638  2018-03-01 00:02:00   853.01  0.0      1.000000
275639  2018-03-01 00:03:00   852.97  0.0      1.000000
275640  2018-03-01 00:04:00   851.00  0.0      1.000000
...                     ...      ...  ...           ...
2638630 2022-08-31 23:56:00  1553.80  1.0    242.547027
2638631 2022-08-31 23:57:00  1554.28  1.0    242.621999
2638632 2022-08-31 23:58:00  1555.01  1.0    242.736018
2638633 2022-08-31 23:59:00  1554.10  1.0    242.593885
2638634 2022-09-01 00:00:00  1553.04  1.0    242.282780

[2362999 rows x 4 columns]


In [12]:
err = df_ret['equity'] / df_xbx['equity_curve']  - 1
err.describe()

count    2.362999e+06
mean     2.423074e-04
std      1.199977e-04
min     -1.066092e-03
25%      1.486942e-04
50%      2.772150e-04
75%      3.066479e-04
max      2.940089e-03
dtype: float64

In [8]:
%%time

df_grid = run_gridsearch(stra_module=STRA, 
                         candle_paths=ETH_PATHS, 
                         contract_type='futures', 
                         simulator_params=SIMULATOR_PARAMS, 
                         start_date=START_DATE, 
                         end_date=END_DATE, 
                         init_capital=INIT_CAPITAL, 
                         face_value=FACE_VALUE, 
                         n_proc=16)

df_grid = df_grid.sort_values('equity', ascending=False, ignore_index=True)
print(df_grid.head())

        equity  itl    n    b  leverage  face_value
0  2069.809571   1h   75  1.5       1.5       0.001
1  2049.436361  30m  150  1.5       1.5       0.001
2  1891.993335  30m  145  1.5       1.5       0.001
3  1839.323227   1h   70  1.5       1.5       0.001
4  1591.957325   1h  100  1.8       1.5       0.001
CPU times: user 67.3 ms, sys: 589 ms, total: 657 ms
Wall time: 3min 50s


In [18]:
%%time

fparams_list = STRA.get_default_factor_params_list()
sparams_list = STRA.get_default_strategy_params_list()

def _run(factor_params, strategy_params):
    leverage = strategy_params['leverage']
    candles = load_candles(ETH_PATHS)
    df_xbx = backtest(candles, factor_params, leverage)
    ret = {'equity': df_xbx['equity_curve'].iat[-1]}
    ret.update(factor_params)
    ret.update(strategy_params)
    return ret

results = Parallel(n_jobs=16)(delayed(_run)(fp, sp) for fp in fparams_list for sp in sparams_list)
df_grid_xbx = pd.DataFrame.from_records(results)

CPU times: user 4.08 s, sys: 390 ms, total: 4.48 s
Wall time: 23min 4s


In [29]:
df_grid_xbx = df_grid_xbx.sort_values('equity', ascending=False, ignore_index=True)
join_idx = ['itl', 'n', 'b', 'leverage']
tmp = df_grid_xbx.join(df_grid.set_index(join_idx), on=join_idx, rsuffix='_neo')
tmp = tmp[tmp['equity'] > 1]
(tmp['equity_neo'] / tmp['equity'] - 1).describe()

count    1931.000000
mean        0.000382
std         0.000753
min        -0.003282
25%        -0.000108
50%         0.000330
75%         0.000816
max         0.003698
dtype: float64